# Вариант парсинга данных для обучения 

In [ ]:
from openai import OpenAI
import threading


questions = []
answers = []


client = OpenAI(api_key='')

def gpt_get():
    for i in range(1000):
        prompt = """I want you to ask yourself 
        a random question or a promt on any topic and then answer on it, length of you answer should be not less then 500 words"""

        completion = client.chat.completions.create(
          model="gpt-3.5-turbo",# gpt4
          messages=[
            {"role": "system", "content": """You a helpfull assistent """},
            {"role": "user", "content": prompt}
          ]
        )

        a = completion.choices[0].message.content

        answers.append(a[a.find('nswer')+6:])
        questions.append(a[9:a.find('nswer')-1])



for i in range(1000):

    thread = threading.Thread(target=gpt_get)
    threads.append(thread)
    thread.start()

df = pd.DataFrame({"Questions":questions,"Answers":answers})

df.to_pickle('promt_data.pkl')

# Архитектура сети

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class LinformerAttention(nn.Module):
    def __init__(self, dim: int, seq_len: int, heads: int, k: int) -> None:
        super().__init__()

        assert dim % heads == 0

        # Stores constant values.
        self.seq_len = seq_len
        self.k = k
        self.heads = heads
        self.dim_head = dim // heads

        # Similar to the original transformer implementation, but with two
        # extra parameters for projecting from the full sequence length.
        self.to_q = nn.Linear(dim, dim, bias=False)
        self.to_k = nn.Linear(dim, dim, bias=False)
        self.to_v = nn.Linear(dim, dim, bias=False)
        self.proj_k = nn.Parameter(torch.randn(seq_len, k))
        self.proj_v = nn.Parameter(torch.randn(seq_len, k))
        self.to_out = nn.Linear(dim, dim)

    def forward(self, x):
        (b, n, d), d_h, h, k = x.shape, self.dim_head, self.heads, self.k

       
        queries = self.to_q(x)
        keys = self.to_k(x)
        values = self.to_v(x)

        
        keys = torch.einsum("bnd,nk->bkd", keys, self.proj_k)
        values = torch.einsum("bnd,nk->bkd", values, self.proj_v)

        
        queries = queries.reshape(b, n, h, -1).transpose(1, 2)  # (B, N, D) -> (B, H, N, D // H)
        keys = keys.reshape(b, k, h, -1).transpose(1, 2)  # (B, K, D) -> (B, H, K, D // H)
        values = values.reshape(b, k, h, -1).transpose(1, 2)  # (B, K, D) -> (B, H, K, D // H)

        out,attention = scale_dot_product(queries, keys, values)

        
        out = out.transpose(1, 2).reshape(b, n, -1)  # (B, H, N, D // H) -> (B, N, D)

        return self.to_out(out)

In [5]:
def scale_dot_product(q,k,v):
    L, S = q.size(-2), k.size(-2)
    d_k = q.size(-1)
    scaled = q@k.transpose(-1,-2)*d_k**(-0.5)# 200x512x512

    scaled = scaled.masked_fill(torch.tril(torch.ones(L, S)).to(device) == 0, float('-inf'))
    attention = F.softmax(scaled, dim=-1)
    values = attention@v
    return values,attention

In [15]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.set_default_dtype(torch.float32)


def scale_dot_product(q,k,v):
    L, S = q.size(-2), k.size(-2)
    d_k = q.size(-1)
    scaled = q@k.transpose(-1,-2)*d_k**(-0.5)# 200x512x512
    #print(scaled.shape)
    scaled = scaled.masked_fill(torch.tril(torch.ones(L, S)).to(device) == 0, float('-inf'))
    attention = F.softmax(scaled, dim=-1)
    values = attention@v
    return values,attention


class PositionwiseFeedForward(nn.Module):
    def __init__(self,d_model, ffn_hidden , drop_prob):
        super().__init__()
        self.linear1 = nn.Linear(d_model,ffn_hidden)
        self.dropout1 = nn.Dropout(drop_prob)
        self.gelu = nn.GELU()
        self.linear2 = nn.Linear(ffn_hidden, d_model)
        
    def forward(self,x):
        x = self.dropout1(self.gelu(self.linear1(x)))
        return self.linear2(x)
    
class MultiHeadAttention(nn.Module):
    def __init__(self,d_model,n_heads):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.head_dim = d_model//n_heads
        self.qkv = nn.Linear(d_model, d_model*3)
        self.linear = nn.Linear(d_model, d_model)
        
    def forward(self,x):
        batch_size, sequence_length, d_model = x.size()
        qkv = self.qkv(x)
        qkv = qkv.reshape(batch_size,sequence_length,self.n_heads, self.head_dim*3).permute(0,2,1,3)
        q,k,v = qkv.chunk(3,dim=-1)#
        values, attention = scale_dot_product(q,k,v)
        
        values = values.permute(0,2,1,3).reshape(batch_size,sequence_length,d_model)
        return self.linear(values)

class Block(nn.Module):
    def __init__(self,d_model, n_heads,drop_prob,max_len,linear):
        super().__init__()
        if linear:
            self.attention = LinformerAttention(d_model, seq_len = max_len, heads = n_heads, k=max_len//2)
        else:
            self.attention = MultiHeadAttention(d_model,n_heads)
        self.norm1 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, ffn_hidden=d_model*3, drop_prob=drop_prob)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout2 = nn.Dropout(p=drop_prob)
        
    def forward(self, x):# 64x200x512

        x = x + self.attention(self.norm1(self.dropout(x)))
        x = x + self.attention(self.norm1(self.dropout(x)))
        x = x + self.ffn(self.norm2(self.dropout(x)))
        return x

In [16]:
import torch.utils.checkpoint as checkpoint

class ScratchTransformer(nn.Module):
    def __init__(self,d_model,n_layers,n_heads,dropout,max_len,linear=False):
        super().__init__()
        self.d_model = d_model
        self.pos_vect = torch.LongTensor([i for i in range(max_len)]).to(device)
        
        self.token_embedding_table = nn.Embedding(tokenizer.vocab_size, d_model, padding_idx=0).to(device)
        self.pos_embedding_table = nn.Embedding(max_len, d_model).to(device)
        self.blocks = nn.Sequential(*[Block(d_model, n_heads=n_heads,drop_prob=dropout,max_len= max_len,linear=linear) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(d_model) # final layer norm
        self.lm_head = nn.Linear(d_model, tokenizer.vocab_size)
                                          
                                                    
    def forward(self,x):
        

        x = self.token_embedding_table(x) +  self.pos_embedding_table(self.pos_vect)
        
        def run_checkpoint(x):
            x = self.blocks(x)
            return x
        
        x = checkpoint.checkpoint(run_checkpoint, x)
        
        x = self.ln_f(x)
        x = self.lm_head(x)
        b,s,d = x.shape
        return x.view(b*s,d)


def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        nn.init.constant_(m.bias, 0.1)


# Wiki-dataset

In [8]:
from datasets import load_dataset

wiki_dataset = load_dataset('wikipedia','20220301.en')

C:\Users\ga232\anaconda3\Lib\site-packages\datasets\load.py:1454: FutureWarning: The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [9]:
from transformers import BertTokenizerFast as BertTokenizer

BERT_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

In [39]:
device = 'cuda'
model = ScratchTransformer(64,8,8,0.1,500,True)
model = model.to('cuda')

In [43]:
%%time
#model(torch.tensor([[i for i in range(500)]]).cuda())

CPU times: total: 0 ns
Wall time: 0 ns


In [44]:
def chunk_func(data):
    chunks = []
    
    dataa = data['text'].split()
    for i in range((len(dataa)//350) -1):
        chunks.append(dataa[i*350:(i+1)*350:])
        
    chunks.append(dataa[(len(dataa)//350)*350:])
    chunks = list(map(lambda x: ' '.join(x),chunks))

    func = lambda x:tokenizer.encode_plus(
      x,
      add_special_tokens=True,
      max_length=500,
      return_token_type_ids=False,
      padding="max_length",
      return_attention_mask=True,
      #return_tensors='pt',
    )["input_ids"][:500] #[0][:500] 

    return list(map(func,chunks))

In [45]:
optimizer = torch.optim.RAdam(model.parameters(),
                 lr = 3e-4)

In [46]:
model

ScratchTransformer(
  (token_embedding_table): Embedding(28996, 64, padding_idx=0)
  (pos_embedding_table): Embedding(500, 64)
  (blocks): Sequential(
    (0): Block(
      (attention): LinformerAttention(
        (to_q): Linear(in_features=64, out_features=64, bias=False)
        (to_k): Linear(in_features=64, out_features=64, bias=False)
        (to_v): Linear(in_features=64, out_features=64, bias=False)
        (to_out): Linear(in_features=64, out_features=64, bias=True)
      )
      (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (ffn): PositionwiseFeedForward(
        (linear1): Linear(in_features=64, out_features=192, bias=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (gelu): GELU(approximate='none')
        (linear2): Linear(in_features=192, out_features=64, bias=True)
      )
      (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (dropout2): Dropout(p=0.1, inplace=False)
   

In [ ]:
from tqdm import tqdm
from IPython.display import clear_output


gradient_accumulation_steps = 10
num_batches = 0

for n,i in enumerate(tqdm(wiki_dataset['train'])):
    chunks = chunk_func(i)

    x,y = torch.tensor(chunks), torch.cat((torch.tensor(chunks)[:,1:],torch.LongTensor([[0] for i in range(len(chunks))])),dim=1)

    x, y = x.to(device), y.to(device)


    # evaluate the loss
    pred = model(x)
    loss = F.cross_entropy(pred, y.reshape(-1))


    loss = loss / gradient_accumulation_steps
    loss.backward()
    num_batches+=1
    
    optimizer.step()
    
    optimizer.zero_grad()

    if n%100==0:
        clear_output()
        print(loss)

# Дообучение итоговой модели на promt'ах GPT-4

In [ ]:
from datasets import load_dataset

a = load_dataset("vicgalle/alpaca-gpt4", 'default',)

In [ ]:
import pandas as pd
df = pd.DataFrame(a['train'])

In [ ]:
data["instruction"] = data["instruction"].apply(lambda x:tokenizer.encode_plus(
  x,
  add_special_tokens=True,
  max_length=500,
  return_token_type_ids=False,
  padding="max_length",
  return_attention_mask=True,
  return_tensors='pt',
)["input_ids"][0][:500])

In [ ]:
data['output'] = data["output"].apply(lambda x:tokenizer.encode_plus(
  x,
  add_special_tokens=True,
  max_length=500,
  return_token_type_ids=False,
  padding="max_length",
  return_attention_mask=True,
  return_tensors='pt',
)["input_ids"][0][:500])

In [ ]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, data):
        #self.data = data.reset_index().drop("index",axis=1).to_numpy()
        self.X = data["instruction"].to_numpy()
        self.y = data['output'].to_numpy()
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return  self.X[idx], self.y[idx],

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(data,train_size = 0.8,random_state=42)

In [ ]:
train_dataset = TextDataset(train_df)
test_dataset = TextDataset(test_df)

In [ ]:
train_loader = DataLoader(train_dataset,batch_size=32,)#pin_memory = True)
valid_loader = DataLoader(test_dataset,batch_size=64,)#pin_memory = True)

In [ ]:
optimizer = torch.optim.RAdam(model.parameters(),
                 lr = 3e-4)

In [ ]:
from IPython.display import clear_output
from tqdm import tqdm
from IPython.display import clear_output

loss_sum = 0
gradient_accumulation_steps = 2
for epoch in tqdm(range(100)):
    for i,(x,y) in enumerate(tqdm(train_loader)):
        
        x, y = x.to(device), y.to(device)


        
        pred = model(x)
        loss = F.cross_entropy(pred, y.reshape(-1))


        loss = loss / gradient_accumulation_steps
        loss.backward()
        if (i + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            
            optimizer.zero_grad()
            break
        if i%100==0:
            clear_output()
            print(loss)